In [3]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [4]:
############### Load the dataset ###################
path_train = 'all_train.csv'
train_data=pd.read_csv(path_train)
samples = train_data.sample(frac=0.1)

In [5]:
X =samples.iloc[:,1:]
Y =samples.iloc[:,0]
classes = 2
Y = to_categorical(Y,classes)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(560000, 28) (560000, 2)
(140000, 28) (140000, 2)


In [7]:
s = MinMaxScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)
y_train = np.argmax(Y_train,axis=-1)
y_test = np.argmax(Y_test,axis=-1)
print(y_train.shape)

(560000,)


In [11]:
from tab2img.converter import Tab2Img
model = Tab2Img()
train = model.fit_transform(X_train, y_train)
test = model.transform(X_test)
print(train.shape,test.shape)

(560000, 6, 6) (140000, 6, 6)


In [12]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(train)):
    img = Image.fromarray(train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(test)):
    img = Image.fromarray(test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(560000, 32, 32, 3) (140000, 32, 32, 3)


In [14]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/20
2188/2188 [==============================] - 162s 74ms/step - loss: 0.3121 - accuracy: 0.8542 - val_loss: 0.4206 - val_accuracy: 0.8093
Epoch 2/20
2188/2188 [==============================] - 158s 72ms/step - loss: 0.2962 - accuracy: 0.8637 - val_loss: 0.3315 - val_accuracy: 0.8456
Epoch 3/20
2188/2188 [==============================] - 159s 73ms/step - loss: 0.2940 - accuracy: 0.8650 - val_loss: 0.2900 - val_accuracy: 0.8655
Epoch 4/20
2188/2188 [==============================] - 160s 73ms/step - loss: 0.2872 - accuracy: 0.8685 - val_loss: 0.2834 - val_accuracy: 0.8701
Epoch 5/20
2188/2188 [==============================] - 160s 73ms/step - loss: 0.2835 - accuracy: 0.8704 - val_loss: 0.2884 - val_accuracy: 0.8669
Epoch 6/20
2188/2188 [==============================] - 158s 72ms/step - loss: 0.2836 - accuracy: 0.8704 - val_loss: 0.2783 - val_accuracy: 0.8741
Epoch 7/20
2188/2188 [==============================] - 159s 73ms/step - loss: 0.2767 - accuracy: 0.8740 - val_loss: 0

In [15]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

17500/17500 [==============================] - 153s 9ms/step - loss: 0.2075 - accuracy: 0.9089
training accuracy:  0.9088892936706543
testing accuracy:  0.8714785575866699
